In [1]:
from gym import Env
from gym.spaces import Discrete, Box, Dict
import numpy as np
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from py4j.java_gateway import JavaGateway

In [26]:
gateway = JavaGateway()
minitruco_java = gateway.entry_point 

In [ ]:
class MinitrucoEnv(Env):
    def __init__(self):
        # Ações que podemos tomar: jogar a carta 0, a carta 1 ou a carta 2
        self.action_space = Discrete(3)
        
        # Estados possíveis
        # - posições: 1=inferior, 2=direita, 3=superior, 4=esquerda    
        # - equipes: 1=posições 1 e 3; 2=posições 2 e 4
        # - cartas: podem valer -1 (null), 0 (fechada) ou um valor de 1
        #           a 14, conforme o valor relativo delas (cartas normais de 1
        #           a 10, manilhas de 11 a 14)
        # - rodadas: 1 a 3
        # - resultado da rodada: a equipe que venceu (1 ou 2), 3 para empate ou -1 para rodada não conlcuída
        # - booleanos (ex.: podeFechada) são 0 ou 1
        # TBD posJogadorPedindoAumento (acho que não zera depois do aumento)
        # TBD tento mineiro (talvez só varie as recompensas, mas é preciso especificar)
        # TBD baralho limpo (provavelmente só vamos excluir o range 1-4)
        self.observation_space = Tuple((
                spaces.Discrete(4, start=1),    # posJogador
                spaces.Discrete(3, start=1),    # numRodadaAtual
                spaces.Discrete(3, start=1),    # resultadoRodada1
                spaces.Discrete(3, start=1),    # resultadoRodada2
                spaces.Discrete(12, start=1),   # valorMao
                spaces.Discrete(13, start=0),   # valorProximaAposta
                spaces.Discrete(4, start=1),    # posJogadorPedindoAumento
                spaces.Discrete(4, start=1),    # posJogadorQueAbriuRodada
                spaces.Discrete(24, start=0),   # pontosEquipe1
                spaces.Discrete(24, start=0),   # pontosEquipe2
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada1Pos1
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada1Pos2
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada1Pos3
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada1Pos4
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada2Pos1
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada2Pos2
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada2Pos3
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada2Pos4
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada3Pos1
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada3Pos2
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada3Pos3
                spaces.Discrete(16, start=-1),  # cartaJogadaRodada3Pos4
                spaces.Discrete(16, start=-1),  # carta1Jogador
                spaces.Discrete(16, start=-1),  # carta2Jogador
                spaces.Discrete(16, start=-1),  # carta3Jogador
                spaces.Discrete(2),             # baralhoSujo
                spaces.Discrete(2)              # podeFechada
        ))
        
        self.episodio = minitruco_java.novoEpisodio()
        return self.episodio.proximoEstado().split()
        
    def step(self, action):
        self.episodio.joga(action)
        observation = self.episodio.proximoEstado().split()
        reward = 0 # TBD recompensa        
        done = False # TBD
        
        return observation, reward, done, {}

    def render(self):
        pass
    
    def reset(self):
        self.episodio.finaliza()
        
        self.episodio = minitruco_java.novoEpisodio()
        return self.episodio.proximoEstado().split()
    

In [104]:
gateway = JavaGateway()
minitruco_java = gateway.entry_point 
episodio = minitruco_java.novoEpisodio()


1 1 -1 -1 1 0 0 1 0 0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 7 9 14 1 0


In [113]:
situacao = episodio.proximoEstado()
print(situacao)

1 2 2 -1 1 0 0 4 1 0 4 6 3 12 -1 -1 -1 3 -1 -1 -1 -1 6 4 4 1 0


In [112]:
episodio2.joga(2)

In [103]:
episodio2.finaliza()

In [109]:
episodio2 = minitruco_java.novoEpisodio()
situacao2 = episodio2.proximoEstado()
print(situacao2)

1 1 -1 -1 1 0 0 2 1 0 -1 6 3 12 -1 -1 -1 -1 -1 -1 -1 -1 6 4 4 1 0


In [115]:
episodio2.joga(1)
situacao2 = episodio2.proximoEstado()
print(situacao2)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/chesterbr/.pyenv/versions/3.7.17/lib/python3.7/site-packages/py4j/java_gateway.py", line 1224, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/chesterbr/.pyenv/versions/3.7.17/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/chesterbr/.pyenv/versions/3.7.17/lib/python3.7/site-packages/py4j/java_gateway.py", line 1229, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o1.proximoEstado

In [17]:
situacao.__class__

py4j.java_collections.JavaMap

In [19]:
#playpen
from gym.spaces import Box, Dict, Discrete, MultiBinary, MultiDiscrete

d = Dict(

    {

        "ext_controller": MultiDiscrete([5, 2, 2]),

        "inner_state": Dict(

            {

                "charge": Discrete(100),

                "system_checks": MultiBinary(10),

                "job_status": Dict(

                    {

                        "task": Discrete(5),

                        "progress": Box(low=0, high=100, shape=()),

                    }

                ),

            }

        ),

    }

)
d

Dict('ext_controller': MultiDiscrete([5 2 2]), 'inner_state': Dict('charge': Discrete(100), 'job_status': Dict('progress': Box(0.0, 100.0, (), float32), 'task': Discrete(5)), 'system_checks': MultiBinary(10)))

In [20]:
        from gym.wrappers import FilterObservation, FlattenObservation
FlattenObservation(dict)

AttributeError: type object 'dict' has no attribute 'observation_space'